In [ ]:
#!pip install pickledb==0.9.2 torch==1.10.2 torchcontrib==0.0.2

In [ ]:
features = "/home/jovyan/data/features.json"
target = "/home/jovyan/data/target.json"
job_name = ""
study_yaml = None

In [ ]:
import pandas as pd
import yaml
from sklearn.preprocessing import LabelEncoder

import neural_network.n_network as nn
from hddataclient import DataRepo

#### Load yaml

In [ ]:
with open(study_yaml) as fh:
    my_study_yaml = yaml.safe_load(fh)

#### Load data - set to pandas DataFrame

In [ ]:
datarepo = DataRepo(
    description="Study as Data Repo",
    volume_name=f"/home/jovyan/_jobs/{job_name}/",
)

In [ ]:
X_df = datarepo.load_dataset(features)
y_df = datarepo.load_dataset(target)

In [ ]:
drop_cols = list(X_df.dtypes.index[X_df.dtypes == "object"])
X = X_df.drop(drop_cols, axis=1)

In [ ]:
response_variable = my_study_yaml["training"]["data"]["target"]["response_variable"]
y_resp = y_df[response_variable]
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y_resp))

#### Default neural network hyperparams

In [ ]:
hyper_param = {
    "learning_rate": (1e-4, 1e-2),
    "bach_norm": [True, False],
    "weight_decay": [0, 0.00001, 0.00005, 0.00009, 0.0001, 0.0005, 0.0009, 0.001,
                     0.005, 0.009, 0.01, 0.05, 0.09,0.1, 0.5, 0.9],
    "snapshot_ensemble": [False],
    "drop_out": (0, 0.9)
}

#### Build model object

In [ ]:
model = nn.NeuralNetwork(
    hyper_param=hyper_param,
    trials=3,
    n_jobs=-1,
    regression=False,
    epochs=20
)

#### Fit model

In [ ]:
output = model.fit(x=X, y=y)

#### Look at model outputs

In [ ]:
output.keys()

#### Look at Optuna results

In [ ]:
output['study'].trials_dataframe().sort_values(by='value')